In [ ]:
#Sorry for this crazy import cell... this will be cleaned up

import psycopg2
import numpy
import nltk
import unicodedata
import pandas as pd
import pprint
import pickle
import re
import os
from nltk.corpus import wordnet
import time
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import nltk.corpus
from nltk.text import TextCollection
import sklearn
import scipy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin

stops = nltk.corpus.stopwords.words('english')
wnl = nltk.WordNetLemmatizer()
nltk.download('averaged_perceptron_tagger')

https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

In [ ]:
#Input your PostGres credentials to connect

dbname = 'yelp'
username = 'postgres'
host = 'localhost'
password = 'Khoobam1234!'

conn = psycopg2.connect('dbname={} user={} host={} password={}'.format(dbname, username, host, password))
cur = conn.cursor()

In [ ]:
cur = conn.cursor()
cur.execute("""
    SELECT * FROM review LIMIT 10
""")

cols = ['review_id', 'user_id', 'business_id', 'stars', 'review_date', 'review_text', 'useful', 'funny', 'cool']

review_sample = pd.DataFrame(cur.fetchall(), columns=cols)

In [ ]:
review_sample

Contractions:
https://gist.github.com/J3RN/ed7b420a6ea1d5bd6d06

In [ ]:
common_nonneg_contr = ["could've",
"he'd",
"he'd've",
"he'll",
"he's",
"how'd",
"how'll",
"how's",
"i'd",
"i'd've",
"i'll",
"i'm",
"i've",
"it'd",
"it'd've",
"it'll",
"it's",
"let's",
"ma'am",
"might've",
"must've",
"o'clock",
"'ow's'at",
"she'd",
"she'd've",
"she'll",
"she's",
"should've",
"somebody'd",
"somebody'd've",
"somebody'll",
"somebody's",
"someone'd",
"someone'd've",
"someone'll",
"someone's",
"something'd",
"something'd've",
"something'll",
"something's",
"that'll",
"that's",
"there'd",
"there'd've",
"there're",
"there's",
"they'd",
"they'd've",
"they'll",
"they're",
"they've",
"'twas",
"we'd",
"we'd've",
"we'll",
"we're",
"we've",
"what'll",
"what're",
"what's",
"what've",
"when's",
"where'd",
"where's",
"where've",
"who'd",
"who'd've",
"who'll",
"who're",
"who's",
"who've",
"why'll",
"why're",
"why's",
"would've",
"y'all",
"y'all'll",
"y'all'd've",
"you'd",
"you'd've",
"you'll",
"you're",
"you've"]

In [ ]:
neg_stops1 = ['no',
 'nor',
 'not',
 'don',
 "don't",
 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't",
 'but',
 "don'",
 "ain't"]

words_to_be_added = ['us']
stops1 = ['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don't",
 'should',
 "should've",
 'now',
 'd',
 'll',
 'm',
 'o',
 're',
 've',
 'y',
 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'ma',
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't"]


In [ ]:
for x in neg_stops1:
    if x in stops1:
        stops1.remove(x)
        
stops1

In [ ]:
new_stops = stops1 + common_nonneg_contr
list(set(new_stops))

# Cell below creates a class that will be used in the vectorization code a few cells down. FYI, path in "_load_words" is machine specific. Also, you have to download the GloVe training set from GloVe website.

In [ ]:
class MeanEmbeddingTransformer(TransformerMixin):
    
    def __init__(self):
        self._vocab, self._E = self._load_words()
        
    
    def _load_words(self):
        E = {}
        vocab = []

        with open(r'PUT YOUR PATH HERE', encoding="utf8") as file:
            for i, line in enumerate(file):
                l = line.split(' ')
                if l[0].isalpha():
                    v = [float(i) for i in l[1:]]
                    E[l[0]] = np.array(v)
                    vocab.append(l[0])
        return np.array(vocab), E            

    
    def _get_word(self, v):
        for i, emb in enumerate(self._E):
            if np.array_equal(emb, v):
                return self._vocab[i]
        return None
    #in _doc_mean Ithink maybe we should get rid of w.lower.strip
    def _doc_mean(self, doc):
        return np.mean(np.array([self._E[w.lower().strip()] for w in doc if w.lower().strip() in self._E]), axis=0)
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return np.array([self._doc_mean(doc) for doc in X])
    
    def fit_transform(self, X, y=None):
        return self.fit(X).transform(X)
    
    
#retrived from https://www.kaggle.com/nhrade/text-classification-using-word-embeddings

In [ ]:
#fyi, tweaked Alice's code here to allow map_on_column to accept another function
def map_on_column(data, function):
    data['review_text'] = data['review_text'].map(lambda x: function(x))
    return data

# Cell below is preproccessing and tokenezation

In [ ]:
# def _remove_stops(tokens):
    stops = nltk.corpus.stopwords.words('english')
    neg_stops = [
    'no', 'not', 'nor', 'don\'', 'don\'t', 'ain', 
    'ain\'t', 'aren\'t', 'aren', 'couldn', 'couldn\'t', 
    'didn', 'didn\'t', 'doesn', 'doesn\'t', 'hadn', 
    'hadn\'t', 'hasn', 'hasn\'t', 'haven', 'haven\'t',
    'isn', 'isn\'t', 'mightn', 'mightn\'t', 'mustn', 
    'mustn\'t', 'needn', 'needn\'t', 'shan', 'shan\'t',
    'shouldn', 'shouldn\'t', 'wasn', 'wasn\'t', 'weren',
    'weren\'t', 'won', 'won\'t', 'wouldn', 'wouldn\'t'
    ]
#still leaves in but and don.. fix this.. 
#doesn't get rid of other obvious stopwords, like i'm, they're....
    for x in neg_stops:
        if x in stops:
            stops.remove(x)
            
    tokens_without_stops = [token for token in tokens if token not in stops]
    return tokens_without_stops       
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].lower()
    tag_dict = {"a": wordnet.ADJ,
                "n": wordnet.NOUN,
                "v": wordnet.VERB,
                "r": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


def _clean_review2(text):
    text = text.lower()
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf8', 'ignore')
    text = re.sub(r"[^A-Za-z\s']", '', text)  
    tokens = [token for token in text.split() if token not in new_stops]
    for i, token in enumerate(tokens):
        tokens[i] = wnl.lemmatize(token, pos= get_wordnet_pos(token))
    return tokens

def _process_review2(text):
    tokens = _remove_stops(_clean_review2(text))
    return tokens




# Cell below is vectorization. So far it can do TFIDF on specified number of n-grams, and I think the part with GloVe word embedding is almost functioning, but I haven't started "word2vec" vectorization yet.

In [ ]:
def dummy_fun(doc):
    return doc

def GloVeDistributedRep(text):
    GloVeVector = numpy.append(MeanEmbeddingTransformer().fit_transform(text))
    return GloVeVector


def vectorize(text, method, grams):
    if method == 'TFIDF':
        tfidf = TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None, ngram_range = (1,grams))
        TDIFvectors = tfidf.fit_transform(map_on_column(text, _clean_review2))
        print(TDIFvectors)
    if method == 'DistributedRep':
        GloVevectors = GloVeDistributedRep(map_on_column(text, _clean_review2))
        print(GloVevectors)
    if method == 'Word2Vec':
        print('not yet defined')
    else:
        print('choose vectorization method')
        

def vectorize2(text, method, grams):
    if method == 'TFIDF':
        tfidf = TfidfVectorizer(analyzer='word', tokenizer=dummy_fun, preprocessor=dummy_fun, token_pattern=None, ngram_range = (1,grams))
        TDIFvectors = tfidf.fit_transform(text)
        print(TDIFvectors)
    if method == 'DistributedRep':
        GloVevectors = GloVeDistributedRep(text)
        print(GloVevectors)
    if method == 'Word2Vec':
        print('not yet defined')
    else:
        print('choose vectorization method')

# Vectorization tests. FYI, takes unprocessed text so after you run one you must rerun query in line 3 to get text back to unprocessed.

In [ ]:
#this one works, or atleast I think it does. The output seems to be right. As you will notice, as you increase ngrams the size of the array increases but the value of each key decreases.
vectorize(review_sample, 'TFIDF', 5)

In [ ]:
#I think this works but I was having a path issue on my comp so I don't know yet. So, it's not working 
#but I think that's just cause of a path issue in the class.
vectorize(review_sample, 'DistributedRep', 1)


In [ ]:
#this is pretty technical. haven't started yet.
vectorize(review_sample['review_text'], 'Word2Vec', 1)

# So the cells below are about n-gram creation. i couldn't get them all to work, but this may be moot work because as you can see above n-grams can be made on the fly within sklearn's TFIDF tool....... the word embeddings don't auto generate ngrams, though so we will probably eventually have to get the below code to work.

In [ ]:
#side note: I couldn't find a way for the n_grams function to work by iterating through an array of a list of unigrams,
#which is what Alice's map/apply function outputs. So I tried doing it this way, which is more in line with what the
#orignal code from stack overflow shows. If it turns out that this approach is too slow or unworkable we may need to find
#antoher way to make ngrams


def uni_to_ngrams(text, n):
        n_grams = ngrams(_clean_review2(text), n)
        return [' '.join(grams) for grams in n_grams]
    
    
def get_grams1(text, n):
    for i in text:
        for r in range(1, n):
            return uni_to_ngrams(text, r)
        
def get_grams2(text, n):
    for i in text:
        return uni_to_ngrams(text, n)

def get_all_grams1(text, n):
    for r in range(1, n):
        return get_grams2(text, r)



In [ ]:
#this works when you feed it a string of text directly, but as you will see later if you try to iterate through the review_text column you get an error.
uni_to_ngrams('Thisoooo isooo aoooo testoooo ooooffff theeee cooodeee hoppeeee itttt wooorrrksss kjaiofad', 5)

In [ ]:
#this does not throw an error, but the point of the code is to return n sets of n-grams of r length, but instead it only returns one set of unigrams.
get_all_grams1('Thisoooo isooo aoooo testoooo ooooffff theeee cooodeee hoppeeee itttt wooorrrksss kjaiofad', 5)

In [ ]:
#this does not work. The error makes me think that I'm misunderstanding something about iteration or the input/output of Alice's _clean_review2 code
get_all_grams1(review_sample['review_text'], 5)

In [ ]:
review_sample['review_text']

In [ ]:
_process_review2()

In [ ]:


import nltk.corpus
from nltk.text import TextCollection

def TFIDFvectorize(corpus):
    corpus = [_clean_review2(doc) for doc in corpus]
    texts = TextCollection(corpus)
    
    for doc in corpus:
        return {
            term: texts.tf_idf(term, doc)
            for term in doc
        }


In [ ]:
#so as the function stands now it works on one review... although the way it's written I thought it would do all reviews.
#also as you will see in next line, when I try to run it on the entie column it does not.
#I think it doesnt work in apply column because TFIDF for each doc relies on the values of other docs
#, so you can't overwrite in place as you iterate
TFIDFvectorize(review_sample['review_text'])